In [26]:
import networkx as nx 
import numpy as np
import json
import pandas as pd
from sage.graphs.graph_input import from_dict_of_lists
import time

In [18]:
def nakljucni_MIS(G, P):
    I = []            #Nastajajoca neodvisna mnozica
    V = list(G.nodes)
    for i in V:        # V tej mnozici izberemo vozlisca, ki jih dodamo neodvisni mnozici. P je permutacija naravnih stevil do vkljucno len(V). Vozlisce dodamo I, ce ima najmanjso vrednost med sosedi.,tako tudi preprecimo ponavljanje vozlisc v mnozici.
        A = []      
        A.append(P[i])
        if list(G.neighbors(i)) != []:
            for j in list(G.neighbors(i)):
                A.append(P[j])
        if P[i] == min(A):
            I.append(i)
    N = [] 
    for i in I:
        N += G.neighbors(i)
    V1 = I + N
    V1 = list(dict.fromkeys(V1)) # V1 je mnozica, ki jo bomo odstranili iz grafa in je sestavljena iz vozlisc iz I, ter njihovih sosedov
    if I == []:
        return []
    else:
        G1 = G.copy()
        G1.remove_nodes_from(V1)
        return list(I + nakljucni_MIS(G1, P))

In [20]:
def tightness(graf, mnozica, v): #Dolocimo "tightness" vozlisca v, v mnozici X, tj. za vsakega soseda vozlisca v, v mnozici graf\X, tightness(v) povecamo za 1
    t = 0
    for w in graf.neighbors(v):
        if w in mnozica:
            t += 1
        else:
            pass
    return t

def lokalno_iskanje(G, I): #Parametra graf v networkx formatu, neodvisna množica I (gre za Python list)
    V = list(G.nodes)
    for v in I:
        L = [] #Gradimo mnozice potencialnih kandidatov za dodajanje v neodvisno mnozico
        for w in G.neighbors(v):
            if G.neighbors(v) != []:
                if tightness(G, I, w) == 1:
                    L.append(w)
                else:
                    pass
            else:
                pass
        if len(L) >= 2:
            for v1 in L:
                for w1 in L:
                    if w1 not in G.neighbors(v1):
                        I.remove(v)
                        I_lokalno = I + [v1, w1]
                        return lokalno_iskanje(G, I_lokalno)
        else:
            return I 

In [21]:
def CLP(G):
    p = MixedIntegerLinearProgram(maximization=True)
    x = p.new_variable(binary=True)
    p.set_objective(sum(x[v] for v in G))

    for u, v in G.edges(labels=False):
        p.add_constraint(x[u] + x[v] <= 1)
    
    p.solve()
    x = p.get_values(x)
    return [v for v, i in x.items() if i]

In [28]:
def zapisi_v_json(datoteka,A):
    with open(datoteka +'.json','w') as js:
        js.write(
            '[' +
            ',\n'.join(json.dumps(i) for i in A) +
            ']\n')

# Dano verjetnost in n
def erdos_renyi(n,p,m,ime_datoteke_json):
    A = []
    for i in range(m):
        G = nx.erdos_renyi_graph(n,p)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]
    #zapisi_v_json(ime_datoteke_json, A)
    return A


# Razlicne n

def erdos_renyi_N(N, p, ime_datoteke_json):
    A = []
    for i in N:
        G = nx.erdos_renyi_graph(i,p)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]
    #zapisi_v_json(ime_datoteke_json, A)
    return A


#Razlicne p

def erdos_renyi_P(n, P, ime_datoteke_json):
    A = []
    for i in P:
        G = nx.erdos_renyi_graph(n,i)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]
    #zapisi_v_json(ime_datoteke_json, A)
    return A


# Ista vozlišča in verjetnost, večkrat


In [22]:
A = []
for i in range(1000):
    G = nx.erdos_renyi_graph(50, 0.3)
    G_slovar = nx.to_dict_of_lists(G)
    A += [G_slovar]

In [27]:
np1000_nakljucno = []
casi_np1000_nakljucno = []
resitve = []
for i in A:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    np1000_nakljucno.append(len(nak))
    casi_np1000_nakljucno.append(time2 - time1)
    resitve.append(nak)

TypeError: 'module' object is not callable

In [100]:
np1000_CLP = []
casi_np1000_CLP = []
for i in A:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    np1000_CLP.append(len(rez))
    casi_np1000_CLP.append(time2 - time1)

In [103]:
np1000_lokalno = []
casi_np1000_lokalno = []
for i in range(len(A)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(A[i]), resitve[i])
    time2 = time()
    np1000_lokalno.append(len(rez))
    casi_np1000_lokalno.append(time2 - time1)

In [109]:
data_np1000 = {"Nakljucno":np1000_nakljucno, "Casi nakljucno": casi_np1000_nakljucno,
           "CLP": np1000_CLP, "Casi CLP": casi_np1000_CLP ,
            "Lokalno iskanje": np1000_lokalno, "Casi lokalno": casi_np1000_lokalno}
data_np1000 = pd.DataFrame(data_np1000)
print(data_np1000)



     Nakljucno  Casi nakljucno  CLP  Casi CLP  Lokalno iskanje  Casi lokalno
0            9        0.003369   11  0.156384                9      0.001147
1            8        0.003180   11  0.134443                9      0.000873
2            7        0.003683   11  0.128538                7      0.001180
3            9        0.003485   11  0.128925                9      0.000886
4            7        0.003106   10  0.192122                9      0.000958
..         ...             ...  ...       ...              ...           ...
995          9        0.002117   11  0.153980               10      0.001072
996         10        0.002280   11  0.164795               10      0.000790
997         10        0.002250   11  0.168162               10      0.000735
998          9        0.002342   13  0.097588                9      0.000670
999          9        0.002402   10  0.200561                9      0.000680

[1000 rows x 6 columns]


# Različna vozlišča, ista verjetnost

In [ ]:
N=[1,2,3,4,5]
B = erdos_renyi_N(N, 0.3, 'ime_datoteke_json')

In [ ]:
Np_nakljucno = []
casi_Np_nakljucno = []
resitve = []
for i in B:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    Np_nakljucno.append(len(nak))
    casi_Np_nakljucno.append(time2 - time1)
    resitve.append(nak)

In [ ]:
Np_CLP = []
casi_Np_CLP = []
for i in A:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    Np_CLP.append(len(rez))
    casi_Np_CLP.append(time2 - time1)

In [ ]:
Np_lokalno = []
casi_Np_lokalno = []
for i in range(len(A)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(A[i]), resitve[i])
    time2 = time()
    Np_lokalno.append(len(rez))
    casi_Np_lokalno.append(time2 - time1)

In [ ]:
data_Np = {"Nakljucno":Np_nakljucno, "Casi nakljucno": casi_Np_nakljucno,
           "CLP": Np_CLP, "Casi CLP": casi_Np_CLP ,
            "Lokalno iskanje": Np_lokalno, "Casi lokalno": casi_Np_lokalno}
data_Np = pd.DataFrame(data_Np)
print(data_Np)


# Ista vozlišča, različna verjetnost

In [31]:
P=[0.1,0.2,0.3,0.4,0.5]
B = erdos_renyi_P(5, P, 'ime_datoteke_json')

In [ ]:
nP_nakljucno = []
casi_nP_nakljucno = []
resitve = []
for i in B:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    nP_nakljucno.append(len(nak))
    casi_nP_nakljucno.append(time2 - time1)
    resitve.append(nak)

In [ ]:
nP_CLP = []
casi_nP_CLP = []
for i in A:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    nP_CLP.append(len(rez))
    casi_nP_CLP.append(time2 - time1)

In [ ]:
nP_lokalno = []
casi_nP_lokalno = []
for i in range(len(A)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(A[i]), resitve[i])
    time2 = time()
    nP_lokalno.append(len(rez))
    casi_nP_lokalno.append(time2 - time1)

In [ ]:
data_nP = {"Nakljucno":nP_nakljucno, "Casi nakljucno": casi_nP_nakljucno,
           "CLP": nP_CLP, "Casi CLP": casi_nP_CLP ,
            "Lokalno iskanje": nP_lokalno, "Casi lokalno": casi_nP_okalno}
data_nP = pd.DataFrame(data_nP)
print(data_nP)
